In [22]:
import cv2
import os
import numpy as np
import torch
from IPython.display import clear_output

In [23]:
def load_images_from_folder(folder, flow=False):
    images = []
    for filename in os.listdir(folder):
        if flow:
            img = cv2.imread(os.path.join(folder,filename), -1)
        else:
            img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [24]:
def show(image, name="image"):
    cv2.imshow(name,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [25]:
def mse(imageA, imageB):
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	return err

In [26]:
def flow_transform(array):
    assert(isinstance(array, np.ndarray))
    array = np.transpose(array, (2, 0, 1))
    # handle numpy array
    tensor = torch.from_numpy(array)
    # put it from HWC to CHW format
    return tensor.float()

In [29]:
def flow2rgb(flow, hsv):
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    hsv = np.float32(hsv)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    return rgb


In [30]:
def OpticalFlow(frame1, frame2, display=False, together=False):
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    frame1_gray = cv2.cvtColor(frame1,cv2.COLOR_RGB2GRAY)
    frame2_gray = cv2.cvtColor(frame2,cv2.COLOR_RGB2GRAY)

    flow = cv2.calcOpticalFlowFarneback(frame1_gray,frame2_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    if display:
        rgb = flow2rgb(flow, hsv)
        if not together:
            show(rgb)
        else:
            stacked = np.vstack((frame1,rgb))
            show(stacked)


In [65]:
# hand-crafted transformation
def apply_flow(frame1, frame2, flow):
    # recon = frame2.copy()
    # recon = np.zeros_like(frame2)
    recon = frame1.copy()
    height = len(frame1)
    width = len(frame1[0])

    for h in range(height):
        for w in range(width):
            vec = flow[h,w,:]   # 2D vector containing OF of pixel (h,w)
            if mag(vec) > 0:
                dx = round(vec[0].item()) # x part of vector
                dy = round(vec[1].item()) # y part of vector
                h_new = h+dy if h+dy < height else height-1 # make sure index stays in bounds
                w_new = w+dx if w+dx < width else width-1
                recon[h_new, w_new, :] = frame1[h, w, :]
            else:
                pass

    return recon
    

In [33]:
def load_flow_from_png(flo_file):
    # flo_file = cv2.imread(png_path, -1)
    flo_img = flo_file[:,:,2:0:-1].astype(np.float32)
    invalid = (flo_file[:,:,0] == 0)
    flo_img = flo_img - 32768
    flo_img = flo_img / 64
    flo_img[np.abs(flo_img) < 1e-10] = 1e-10
    flo_img[invalid, :] = 0
    return(flo_img)

In [62]:
def mag(vec):
    return np.sqrt(vec[0]**2+vec[1]**2)

# Loading A & B

In [27]:
A_path = r"C:\Users\20182475\OneDrive - TU Eindhoven\TUe documents\BAU3\Honors - Smart Mobility\DATA\image_20_to_be_converted_to_cv\A"
B_path = r"C:\Users\20182475\OneDrive - TU Eindhoven\TUe documents\BAU3\Honors - Smart Mobility\DATA\image_20_to_be_converted_to_cv\B"
flow_path = r"C:\Users\20182475\OneDrive - TU Eindhoven\TUe documents\BAU3\Honors - Smart Mobility\DATA\flow_20_to_be_converted_to_cv"

In [28]:
A = load_images_from_folder(A_path)
B = load_images_from_folder(B_path)
flow_ = load_images_from_folder(flow_path, flow=True)

In [32]:
i = 12
show(A[i])
show(B[i])
# mse(A[i],B[i])

# Calculate OF

In [10]:
OpticalFlow(A[i], B[i], display=True, together=True)

NameError: name 'i' is not defined

In [31]:
%%time

for i in range(4):
    OpticalFlow(A[i], B[i], display=True, together=True)

Wall time: 20 s


# Apply Transformation

In [52]:
# flow from 3d to 2d
flow = []
for i in range(len(flow_)):
    flow.append(load_flow_from_png(flow_[i]))


In [37]:
%%time
# reconstruct frame2 given frame1 and flow1->2, save them

for i in range(len(A)):
    reconstructed = apply_flow(A[i], B[i], flow[i])
    clear_output(wait=True)
    print(i)
    # cv2.imwrite('reconstructed_using_transform/reconstructed_'+ str(i) + ".png", reconstructed)
    cv2.imshow(i, reconstructed)
    cv2.waitKey(0)

1


KeyboardInterrupt: 

In [61]:
vec = flow[0][200,600]
mag = np.sqrt(vec[0]**2+vec[1]**2)
mag

0.5577052592992109

# Apply flow2rgb for ground truths

In [61]:
for i, fl in enumerate(flow):
    hsv = np.zeros((len(fl), len(fl[0]), 3))
    hsv[...,1] = 255
    rgb = flow2rgb(fl, hsv)
    if i < 3:
        show(rgb)
    cv2.imwrite('groundtruth_rgb/gt_'+ str(144+i) + ".png", rgb)
